In [ ]:
import pandas as pd
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
subs = pd.read_csv("data/auctions.csv", dtype = {"platform": np.int8, "ref_type_id": np.int8, "source_id": np.int8}, usecols = ['auction_type_id', 'date', 'platform', 'source_id'])
#evts = pd.read_csv("data/events.csv", dtype = {"event_id": np.int16, "application_id": np.int16})
#inst = pd.read_csv("data/installs.csv", dtype = {"application_id": np.int16, "click_hash": np.float16})
clks = pd.read_csv("data/clicks.csv.gzip", dtype = {"advertiser_id": np.int8, "action_id": np.float16, "source_id": np.int8, "wifi_connection": bool, "carrier_id": np.float16, "brand": np.float16, "touchX": np.float16, "touchY": np.float16})

# Análisis sobre los clicks

In [ ]:
clks.describe()

In [ ]:
clks.head()

In [ ]:
clks['wifi_connection'].value_counts()

In [ ]:
clks.memory_usage(deep = True).sum()

## ¿Cómo se distribuye el tiempo que tardan los usuarios en dar click?

In [ ]:
clks.count()

In [ ]:
clickTime = clks['timeToClick'].dropna()

In [ ]:
clickTime.count()

In [ ]:
clickTime.nlargest(5)

In [ ]:
grafico = clickTime.plot.hist(bins=50, color='goldenrod', figsize=(10,8))
grafico.set_title("Histograma: cantidad de segundos que tarda un usuario en dar click", fontsize = 20)
grafico.set_xlabel("Tiempo en segundos",fontsize=15)
grafico.set_ylabel("Frecuencia", fontsize=15)

Veamos si aplicando el logaritmo se logra un gráfico más descriptivo:

In [ ]:
clickTimeLog = clickTime.transform(lambda x: np.log(x))

In [ ]:
sns.set_style('darkgrid')
grafico_log = clickTimeLog.plot.hist(bins=50, color='skyblue', figsize=(10,5))
grafico_log.set_title("Histograma: cantidad de segundos (log) que tarda un usuario en dar click", fontsize = 20)
grafico_log.set_xlabel("Tiempo en segundos (log)",fontsize=15)
grafico_log.set_ylabel("Frecuencia", fontsize=15)

## Veamos ahora qué sucede con la posición desde donde se dan los clicks:

In [ ]:
clickPos = clks.loc[:, ['touchX', 'touchY']].dropna()
clickPos.head()

In [ ]:
clickPos['touchX'].value_counts().head()

In [ ]:
clickPos['touchY'].value_counts().head()

Veo si un gráfico de distribución conjunta me muestra algo sobre estos valores

In [ ]:
plt.figure(figsize=(8,8))
g_dist = sns.distplot(clickPos['touchX'], color='crimson', label = 'Posición en X')
g_dist = sns.distplot(clickPos['touchY'], color='royalblue', label = 'posición en Y')
g_dist.set_title("Distribución de la posición de clicks en pantalla", fontsize = 25)
g_dist.set_xlabel("Posición en pantalla", fontsize = 15)
g_dist.set_ylabel("Densidad", fontsize = 15)
g_dist.legend()

Alternativa utilizando un Scatter plot

In [ ]:
plt.figure(figsize = (10,5))
g = sns.regplot(x = "touchX", y = "touchY", 
                    data = clickPos, color = 'orchid')
g.set_title("Relacion entre los clicks en la posición X e Y", fontsize = 20)
g.set_xlabel("Posición en X", fontsize = 15)
g.set_ylabel("Posición en Y", fontsize = 15)

# Análisis sobre las subastas

In [ ]:
subs.describe()

In [ ]:
subs.count()

## Veo que información puedo obtener de las Fechas y horarios en que se realizan las subastas

In [ ]:
subs['day'] = pd.to_datetime(subs['date'], format='%Y%m%d %H:%M:%S.%f').dt.day
subs['hour'] = pd.to_datetime(subs['date'], format='%Y%m%d %H:%M:%S.%f').dt.hour

In [ ]:
subs.head()

In [ ]:
subs_counts = subs.groupby(['day', 'hour']).count()

In [ ]:
subs_counts = subs_counts.reset_index()
subs_counts.head()

In [ ]:
subs_day_hour = subs_counts.pivot_table(index='day', columns='hour', values='date', aggfunc='mean')
subs_day_hour.head()

In [ ]:
plt.figure(figsize = (10,5))
heatMap = sns.heatmap(subs_day_hour,  cmap="PuRd", )
heatMap.set_title("Cantidad Promedio de Subastas por Día y Hora", fontsize = 20)
heatMap.set_xlabel("Hora", fontsize = 15)
heatMap.set_ylabel("Día", fontsize = 15)

## Veo que información encuentro sobre el campo platform

In [ ]:
platformCount = subs['platform'].value_counts().reset_index()
platformCount['platform_name'] = platformCount['index'].apply(lambda x: 'Android' if (x == 1) else 'IOS')
platformCount.columns = ['platform_id', 'platform_count', 'platform_name']
platformCount

In [ ]:
plt.pie(platformCount['platform_count'].to_list(), labels = platformCount['platform_name'].to_list(), autopct='%1.1f%%', colors = ['yellowgreen', 'mediumaquamarine'], shadow=True, radius = 1.7)
plt.legend(platformCount['platform_name'].to_list(), loc = "upper right")
plt.title("Subastas: Plataforma de Dispositivo Utilizadas", {'fontsize': 20,
        'fontweight' : 20,
        'verticalalignment': 'bottom',
        'horizontalalignment': 'center'})
plt.show()

## Quiero averiguar si los source_id más populares coinciden con los que generan más clicks

In [ ]:
source_subs = subs['source_id'].value_counts()
source_subs

In [ ]:
source_clicks = clks['source_id'].value_counts()
source_clicks

Como hay varios source_ids que tienen muy pocos clicks los descarto ya que no aportan mucho, y tomo sólo los 5 ids más populares

In [ ]:
source_clicks = source_clicks.nlargest(5)
source_clicks

Grafico estos valores para ver mejor la relación entre los ids

In [ ]:
plt.figure(figsize = (10,8))
plot_source_subs = sns.barplot(x = source_subs.index, y = source_subs.values, orient = 'v', palette = 'YlOrRd')
plot_source_subs.set_title("Cantidad de Subastas según su source_id", fontsize = '25')
plot_source_subs.set_xlabel("Source_id", fontsize = 15)
plot_source_subs.set_ylabel("Cantidad de Apariciones", fontsize = 15)
for p in plot_source_subs.patches:
    plot_source_subs.annotate(format(p.get_height()), (p.get_x() + p.get_width() / 2.,
                                               p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

Ahora veo que sucede con los source_id de los clicks

In [ ]:
plt.figure(figsize = (10,8))
plot_source_clicks = sns.barplot(x = source_clicks.index, y = source_clicks.values, orient = 'v', palette = 'Spectral')
plot_source_clicks.set_title("Source_ids Más Populares entre los Clicks", fontsize = '25')
plot_source_clicks.set_xlabel("Source_id", fontsize = 15)
plot_source_clicks.set_ylabel("Cantidad de Apariciones", fontsize = 15)
for p in plot_source_clicks.patches:
    plot_source_clicks.annotate(format(p.get_height()), (p.get_x() + p.get_width() / 2.,
                                               p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

## Veo si hay algo interesante en el campo Carrier_id del set de clicks 

In [ ]:
clks.count()

In [ ]:
carriers = clks['carrier_id'].value_counts().to_frame().reset_index()
carriers.columns = ['carrier_id', 'carrier_id_count']

## ¿Habrá fechas en que se generaron más clicks desde ciertos operadores móviles?

In [ ]:
# filtro los datos
carriers_days = clks.loc[:, ['created', 'carrier_id']]
carriers_days['Fecha'] = pd.to_datetime(carriers_days['created']).dt.date
carriers_days.head()

In [ ]:
#como tengo muchoos carrier_ids pero unos pocos concentran la mayoria de los clciks, los filtro
carriers = carriers.loc[carriers['carrier_id_count'] > 1000,:]
carriers.count()

In [ ]:
carriers_days.head()

In [ ]:
#Agrupo por día y operador para ver cuantos clicks tengo cada día segun el operador
days = pd.merge(carriers_days, carriers, on = 'carrier_id', how = 'inner' )
days = days.groupby(['Fecha', 'carrier_id']).agg({'created': 'count'}).reset_index()
days.head()

Pruebo una visualización con un heatmap

In [ ]:
pivot_days = pd.pivot_table(days, columns = 'carrier_id', index = 'Fecha', values = 'created', aggfunc = 'sum').fillna(0)
pivot_days.head()

In [ ]:
plt.figure(figsize = (15,8))
heat_map = sns.heatmap(pivot_days,  cmap = "OrRd",  linewidths = 0.1)
heat_map.set_title("Cantidad de clicks según el día para los carrier_id más populares", fontsize = '20')
heat_map.set_xlabel("Carrier_id", fontsize = 15)
heat_map.set_ylabel("Fecha", fontsize = 15)

De este gráfico lo que logramos ver como el carrier_id 13 tiene alta cantidad de Clicks durante el día 13, y lo mismo se repite para el carrier_id 4 durante el día 11 y 12.
El resto de los carrier_ids son menos frecuentes, aunque también se denota un leve aumento, al menos hasta el día 12.
Otra cosa a remarcar del gráfico es que durante los días 5 y 6 la actividad resgistrada es muy baja, de hecho, esto ya lo hemos visto en otra sección del informe, donde podíamos ver que, a grandes rasgos, la cantidad de clicks iba aumentando en función del tiempo.

# Análisis Sobre Coordenadas

## Me interesa saber cómo se distrubuyen los valores de latitud y longitud desde donde se dan los clicks

In [ ]:
lat_long = clks.filter(items = ['latitude', 'longitude']).dropna()
lat_long.head()

In [ ]:
plt.figure(figsize=(10,5))
lat_dist = sns.distplot(lat_long['latitude'], color = 'purple', label = 'Latitud')
lat_dist.set_title("Distribución de la Latitud desde donde se generan los Clicks", fontsize = 25)
lat_dist.set_xlabel("Latitud", fontsize = 15)
lat_dist.set_ylabel("Densidad", fontsize = 15)


Voe que pasa si utilizo el logaritmo

In [ ]:
lat_long['latitude_log'] = lat_long['longitude'].transform(lambda x : np.log(x))
lat_long.head()

Vuelvo a graficar a ver si mejora la representación

In [ ]:
plt.figure(figsize=(10,5))
lat_dist = sns.distplot(lat_long['latitude_log'], color = 'purple', label = 'Latitud_log')
lat_dist.set_title("Distribución de la Latitud_log desde donde se generan los Clicks", fontsize = 25)
lat_dist.set_xlabel("Latitud_log", fontsize = 15)
lat_dist.set_ylabel("Densidad", fontsize = 15)


## Intento de BaseMap

Importo bibliotecas necesarias para utilizar mapas

In [ ]:
from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

from matplotlib.colors import Normalize
import matplotlib.colors as colors
from numpy import array
from numpy import max

Hago una lista con las latiudes y longitudes

In [ ]:
lats, long = list(clks['latitude']), list(clks['longitude'])

Creo un mapa sin los puntos para chquear

In [ ]:
eq_map = Basemap(projection='robin', resolution = 'l', area_thresh = 1000.0, lat_0=0, lon_0=-130) eq_map.drawcoastlines() eq_map.drawcountries() eq_map.fillcontinents(color = 'gray') eq_map.drawmapboundary() plt.show()


Ahora creo el mismo pero marcando las coordenadas del data frame


In [ ]:
figu, ax = plt.subplots(figsize=(20,10)) eq_map = Basemap(projection='robin', resolution = 'l', area_thresh = 1000.0, lat_0=1.20, lon_0=1.07, llcrnrlon=-2, llcrnrlat=4, urcrnrlon=4, urcrnrlat=0.3) eq_map.drawcoastlines() eq_map.drawcountries() eq_map.fillcontinents(color = 'gray') eq_map.drawmapboundary() eq_map.drawmeridians(np.arange(0, 360, 30)) eq_map.drawparallels(np.arange(-90, 90, 30))

## Coordenadas a posiciones

In [ ]:
x,y = eq_map(long, lats) eq_map.plot(x, y, '^', markersize=6)

Observo que todos los puntos caen en el mismo lugar. Las coordenadas son diferentes pero casi todas provienen casi del mismo lugar. Chequeo esto en el data frame


In [ ]:
clks.head(4)

In [ ]:
coord = clks.loc[:, ['longitude', 'latitude']].dropna() coord.head()

Acerco el mapa en la zona donde caen dichos puntos (Es el Golfo de Guinea en el Atlántico Norte)

In [ ]:
figu, ax = plt.subplots(figsize=(20,10)) mapa2 = Basemap(projection='merc', lat_0 = 0, lon_0 = 0, resolution = 'i', llcrnrlon=-10, llcrnrlat=-10, urcrnrlon=10, urcrnrlat=10) mapa2.drawcoastlines() mapa2.drawcountries() mapa2.fillcontinents(color = 'gray') mapa2.drawmapboundary() mapa2.drawmeridians(np.arange(0, 360, 30)) mapa2.drawparallels(np.arange(-90, 90, 30))

## Coordenadas a posiciones

In [ ]:
x,y = mapa2(long, lats) mapa2.plot(x, y, '^', markersize=6)

Debido a los pocos codigos de pais, podemos decir que no provenian de muchos, sin embargo los datos de la procedencia de los clicks no aportan información contundente.

In [ ]:
clks["country_code"].unique()